# Create the layers for the community and non community protected areas
The WDPA layer is available via the ESRI Living Atlas, it is updated every month and it contains all types of protected areas, both community and non-community types. 
We want to extract the community protected areas using the wildcards: `'%Aborig%','%Indigen%', '%Commun%','%Conservanc%','%Local%', '%Region%','%Trust%','%Conservator%,'%Private%','%Nature Center%'`
The RAISG layer has been uploaded to the Arcgis Online account. Although the original data lives also [here](https://geo.socioambiental.org/webadaptor2/rest/services/raisg/raisg_tis/MapServer/2) but this is a map server, not sure what are the permissions to work with it. 

In [ ]:
import arcgis
from arcgis.gis import GIS
import arcpy
import datetime
import os

In [ ]:
mydate = datetime.datetime.now()
date = mydate.strftime("%y%b")
date

In [ ]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA"

In [ ]:
tmp = []
with open("../.env") as f:
    for line in f:
        tmp.append(line)
aol_password = tmp[0].split("=")[1].split("\n")[0]
aol_username = tmp[1].split("=")[1].split("\n")[0]

In [ ]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [ ]:
unep_wdpa = "ae78aeb913a343d69e950b53e29076f7"
unep_wdpa_item=gis.content.get(unep_wdpa)

So far the structure of the WDPA item is of two layers, one of points and another one of polygons, the one of interest here is: the `WDPA_poly_Latest` layer.

In [ ]:
layerNames = []
for lyr in unep_wdpa_item.layers: 
    layerNames.append(lyr.properties.name)
    #print(lyr.properties.name)
print(layerNames)

In [ ]:
s = "WDPA_poly_Latest"
i = 0
length = len(layerNames)
while i<length:
    if s == layerNames[i]:
        m = i
    i += 1
print(layerNames[m])

In [ ]:
unep_wdpa_url = unep_wdpa_item.layers[m].url

In [ ]:
wdpa_comm_name = f"wdpa_comm_{date}" 
wdpa_not_comm_name = f"wdpa_not_comm_{date}" 

In [ ]:
workingGDB = r"C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb"

In [ ]:
start = datetime.datetime.now()
arcpy.conversion.FeatureClassToFeatureClass(in_features = unep_wdpa_url, 
                                            out_path = workingGDB,
                                            out_name = wdpa_comm_name,
                                            where_clause = "DESIG_ENG LIKE  '%Aborig%' or DESIG_ENG LIKE '%Indigen%'  or DESIG_ENG LIKE '%Commun%' or DESIG_ENG LIKE '%Conservanc%' or DESIG_ENG LIKE '%Local%' or DESIG_ENG LIKE '%Region%' or DESIG_ENG LIKE '%Trust%' or DESIG_ENG LIKE '%Conservator%' or DESIG_ENG LIKE '%Private%' or DESIG_ENG LIKE '%Nature Center%'")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

In [ ]:
start = datetime.datetime.now()
arcpy.conversion.FeatureClassToFeatureClass(in_features = unep_wdpa_url, 
                                            out_path = workingGDB,
                                            out_name = wdpa_not_comm_name,
                                            where_clause = "DESIG_ENG NOT LIKE  '%Aborig%' and DESIG_ENG NOT LIKE '%Indigen%'  and DESIG_ENG NOT LIKE '%Commun%' and DESIG_ENG NOT LIKE '%Conservanc%' and DESIG_ENG NOT LIKE '%Local%' and DESIG_ENG NOT LIKE '%Region%' and DESIG_ENG NOT LIKE '%Trust%' and DESIG_ENG NOT LIKE '%Conservator%' and DESIG_ENG NOT LIKE '%Private%' and DESIG_ENG NOT LIKE '%Nature Center%'")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

In [ ]:
raisg = "f3cd7abb0f014fb09542d8553eba4b25"
raisg_item=gis.content.get(raisg)
layerNamesRAISG = []
for lyr in raisg_item.layers: 
    layerNamesRAISG.append(lyr.properties.name)
    #print(lyr.properties.name)
tis = "Tis_TerritoriosIndigenas"
i = 0
length = len(layerNamesRAISG)
while i<length:
    if tis == layerNamesRAISG[i]:
        mtis = i
    i += 1
print(layerNamesRAISG[mtis])

In [ ]:
tis_raisg_url = raisg_item.layers[mtis].url

# Creating and calculating the `NAME` and `REP_AREA` fields for the RAISG layer has been done in arcgis Online. 

Currently the app uses the fields from the wdpa layers to feed the tooltips, the fields are `NAME` and `REP_AREA`. The latter is the area in km^2. 
Meanwhile the TIs layer has information on the name, which has 85 missing data, and the area in ha. The fields are: `nombre` and `area_sig_h` 
So we have to calculate the RAISG area in km2 and create a new field with the name changing the field's name. 

In [ ]:
wdpa_comm_path = os.path.join(workingGDB, wdpa_comm_name)
out_merge = os.path.join(workingGDB,f"wdpa_raisg_merge_{date}")

# To merge the RAISG and the WDPA comm 
1. Create an empty fc because `Append` modifies the target feature class
2. Create FieldMapping object to manage appended output fields. 

In [ ]:
out_append = f"comm_wdpa_raisg_appended_{date}"

In [ ]:
emptyFC = arcpy.management.CreateFeatureclass(workingGDB, 
                                    out_append, 
                                    "POLYGON", 
                                    wdpa_comm_path)

In [ ]:
fieldMappings = arcpy.FieldMappings()

# Add all fields from both TIs and the community WDPA
fieldMappings.addTable(emptyFC)
fieldMappings.addTable(tis_raisg_url)

# Remove the fields NAME and REP_AREA from the field map (we are going to map how they have to be merged)
for field in fieldMappings.fields:
    if field.name in ["NAME", "REP_AREA"]:
        fieldMappings.removeFieldMap(fieldMappings.findFieldMapIndex(field.name))

# Add input fields "NAME" from both layers into new output field 
fldMap_protName = arcpy.FieldMap()
fldMap_protName.addInputField(wdpa_comm_path, "NAME")
fldMap_protName.addInputField(tis_raisg_url, "NAME")

# Set name of new output field "NAME"
protName = fldMap_protName.outputField
protName.name = "NAME"
fldMap_protName.outputField = protName

# Add output field to field mappings object
fieldMappings.addFieldMap(fldMap_protName)

# Add input fields "REP_AREA" from both layers into new output field 
fldMap_protArea = arcpy.FieldMap()
fldMap_protArea.addInputField(wdpa_comm_path, "REP_AREA")
fldMap_protArea.addInputField(tis_raisg_url, "REP_AREA")

# Set name of new output field "REP_AREA"
protArea = fldMap_protArea.outputField
protArea.name = "REP_AREA"
fldMap_protArea.outputField = protArea

# Add output field to field mappings object
fieldMappings.addFieldMap(fldMap_protArea)

In [ ]:
start = datetime.datetime.now()
arcpy.management.Append([tis_raisg_url, wdpa_comm_path], 
                        emptyFC,
                        "NO_TEST",
                        fieldMappings, '')
end = datetime.datetime.now() - start
print(f"Finished in {end}")

From here we are going to try to publish the data to arcgis online. 